# IMPORTS

In [1]:
import sys
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
sys.path.append(os.path.abspath(".."))       # for 'protonet_STOP_bddoia_modules' folder
sys.path.append(os.path.abspath("../.."))    # for 'data' folder
sys.path.append(os.path.abspath("../../..")) # for 'models' and 'datasets' folders

print(sys.path)

['/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation/bddoia/notebooks', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python38.zip', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8/lib-dynload', '', '/users-1/eleonora/.local/lib/python3.8/site-packages', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8/site-packages', '/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation/bddoia', '/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation', '/users-1/eleonora/reasoning-shortcuts/IXShort']


In [2]:
import csv
import torch
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import datetime
import numpy as np
import setproctitle, socket, uuid

from typing import List

from models import get_model
from models.mnistdpl import MnistDPL
from datasets import get_dataset

from argparse import Namespace
from tqdm import tqdm
from torch.utils.data import DataLoader
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix

from utils import fprint
from utils.status import progress_bar
from utils.metrics import evaluate_metrics
from utils.dpl_loss import ADDMNIST_DPL
from utils.checkpoint import save_model
from torch.utils.data import Dataset, DataLoader

from warmup_scheduler import GradualWarmupScheduler

from backbones.bddoia_protonet import PrototypicalLoss
from protonet_bddoia_modules.data_modules.proto_data import build_prototypical_dataloaders  # TODO: use to refactor
from protonet_bddoia_modules.arguments import args_dpl 
from baseline_modules.supervision_modules.build_sup_set_joint import get_augmented_train_loader  # TODO: use to refactor
from protonet_STOP_bddoia_modules.proto_modules.proto_helpers import (
    assert_inputs,
    get_random_classes,
)
from protonet_STOP_bddoia_modules.proto_modules.proto_functions import (
    train_my_prototypical_network,
)

# SETUP

In [3]:
SEED = 1
UNS_PERCENTAGE = 1.0

In [4]:
args = args_dpl
args.seed = SEED

# logging
args.conf_jobnum = str(uuid.uuid4())
args.conf_timestamp = str(datetime.datetime.now())
args.conf_host = socket.gethostname()

# set job name
setproctitle.setproctitle(
    "{}_{}_{}".format(
        args.model,
        args.buffer_size if "buffer_size" in args else 0,
        args.dataset,
    )
)

# saving
save_folder = "bddoia" 
save_model_name = 'dpl'
save_paths = []
save_path = os.path.join("..",
    "notebook-outputs", 
    save_folder, 
    "my_models", 
    save_model_name,
    f"episodic-proto-net-pipeline-{UNS_PERCENTAGE}-PROVA"
)
save_paths.append(save_path)

print("Seed: " + str(args.seed))
print(f"Save paths: {str(save_paths)}")

Seed: 1
Save paths: ['../notebook-outputs/bddoia/my_models/dpl/episodic-proto-net-pipeline-1.0-PROVA']


# UTILS

## Test Set Evaluation

In [5]:

# * helper function for 'plot_multilabel_confusion_matrix'
def convert_to_categories(elements):
    # Convert vector of 0s and 1s to a single binary representation along the first dimension
    binary_rep = np.apply_along_axis(
        lambda x: "".join(map(str, x)), axis=1, arr=elements
    )
    return np.array([int(x, 2) for x in binary_rep])


# * BBDOIA custom confusion matrix for concepts
def plot_multilabel_confusion_matrix(
    y_true, y_pred, class_names, title, save_path=None
):
    y_true_categories = convert_to_categories(y_true.astype(int))
    y_pred_categories = convert_to_categories(y_pred.astype(int))

    to_rtn_cm = confusion_matrix(y_true_categories, y_pred_categories)

    cm = multilabel_confusion_matrix(y_true, y_pred)
    num_classes = len(class_names)
    num_rows = (num_classes + 4) // 5  # Calculate the number of rows needed

    plt.figure(figsize=(20, 4 * num_rows))  # Adjust the figure size

    for i in range(num_classes):
        plt.subplot(num_rows, 5, i + 1)  # Set the subplot position
        plt.imshow(cm[i], interpolation="nearest", cmap=plt.cm.Blues)
        plt.title(f"Class: {class_names[i]}")
        plt.colorbar()
        tick_marks = np.arange(2)
        plt.xticks(tick_marks, ["0", "1"])
        plt.yticks(tick_marks, ["0", "1"])

        fmt = ".0f"
        thresh = cm[i].max() / 2.0
        for j in range(cm[i].shape[0]):
            for k in range(cm[i].shape[1]):
                plt.text(
                    k,
                    j,
                    format(cm[i][j, k], fmt),
                    ha="center",
                    va="center",
                    color="white" if cm[i][j, k] > thresh else "black",
                )

        plt.ylabel("True label")
        plt.xlabel("Predicted label")

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.suptitle(title)

    if save_path:
        plt.savefig(f"{save_path}_total.png")
    else:
        plt.show()

    plt.close()

    return to_rtn_cm


# * Concept collapse (Soft)
def compute_coverage(confusion_matrix):
    """Compute the coverage of a confusion matrix.

    Essentially this metric is
    """

    max_values = np.max(confusion_matrix, axis=0)
    clipped_values = np.clip(max_values, 0, 1)

    # Redefinition of soft coverage
    coverage = np.sum(clipped_values) / len(clipped_values)

    return coverage


# * BDDOIA custom confusion matrix for actions
def plot_actions_confusion_matrix(c_true, c_pred, title, save_path=None):

    my_scenarios = {
        "forward": [slice(0, 3), slice(0, 3)],  
        "stop": [slice(3, 9), slice(3, 9)],
        "left": [slice(9, 11), slice(18,20)],
        "right": [slice(12, 17), slice(12,17)],
    }

    to_rtn = {}

    # Plot confusion matrix for each scenario
    for scenario, indices in my_scenarios.items():

        g_true = convert_to_categories(c_true[:, indices[0]].astype(int))
        c_pred_scenario = convert_to_categories(c_pred[:, indices[1]].astype(int))

        # Compute confusion matrix
        cm = confusion_matrix(g_true, c_pred_scenario)

        # Plot confusion matrix
        plt.figure()
        plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
        plt.title(f"{title} - {scenario}")
        plt.colorbar()

        n_classes = c_true[:, indices[0]].shape[1]

        tick_marks = np.arange(2**n_classes)
        plt.xticks(tick_marks, ["" for _ in range(len(tick_marks))])
        plt.yticks(tick_marks, ["" for _ in range(len(tick_marks))])

        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        plt.tight_layout()

        # Save or show plot
        if save_path:
            plt.savefig(f"{save_path}_{scenario}.png")
        else:
            plt.show()

        to_rtn.update({scenario: cm})

        plt.close()

    return to_rtn

## Other Utils

In [6]:

# * method used to check if all encoder parameters are registered in the model's optimizer
def check_optimizer_params(model):
    """Check that all encoder parameters are registered in the optimizer."""
    # Get all encoder parameters
    encoder_params = []
    for i in range(21):
        encoder = model.encoder[i]
        for name, param in encoder.named_parameters():
            if not param.requires_grad:
                continue  # skip frozen params
            encoder_params.append((f"encoder_{i}.{name}", param))

    # Get all parameters in the optimizer
    opt_param_ids = set(id(p) for group in model.opt.param_groups for p in group['params'])

    # Check each encoder param is in the optimizer
    missing = [(name, p.shape) for name, p in encoder_params if id(p) not in opt_param_ids]

    if missing:
        print("⚠️ The following parameters are missing from the optimizer:")
        for name, shape in missing:
            print(f"  - {name}: {shape}")
        raise RuntimeError("Some encoder parameters are not registered in the optimizer.")
    else:
        print("✅ All encoder parameters are correctly registered in the optimizer.")

In [7]:

# * semi-deterministic variant of get_random_classes where the positive examples are always the same for a given class index
def get_per_class_support_set(proto_datasets:Dataset, pos_examples:dict, class_idx:int, device:str, debug=False):
    pos_list = pos_examples[class_idx]  
    support_embeddings_pos = torch.stack([ ex['images_embeddings_raw'].unsqueeze(0) for ex in pos_list ], dim=0).to(device)
    support_labels_pos = torch.ones(len(pos_list), dtype=torch.long, device=device)
    num_pos_labels = support_labels_pos.sum().item()
    if debug:
        print(f"Class {class_idx}: {support_labels_pos.shape} embeddings, {support_labels_pos.shape} labels (all 1)")

    proto_labels = proto_datasets[class_idx].labels
    proto_data = proto_datasets[class_idx].embeddings
    
    mask = proto_labels == 0
    proto_data_neg = proto_data[mask]
    proto_labels_neg = proto_labels[mask]
    support_embeddings_neg, support_labels_neg = get_random_classes(
        proto_data_neg, proto_labels_neg, n_support=num_pos_labels, n_classes=1
    )
    if debug:
        print("Support embeddings shape: ", support_embeddings_neg.shape)
        print("Support labels shape: ", support_labels_neg.shape)
    
    assert torch.all(support_labels_neg == 0), "support_labels contains non-zero entries"

    support_embeddings_combined = torch.cat([support_embeddings_pos, support_embeddings_neg], dim=0)
    support_labels_combined = torch.cat([support_labels_pos, support_labels_neg], dim=0)
    if debug:
        print("Combined support embeddings shape:", support_embeddings_combined.shape)
        print("Combined support labels shape:", support_labels_combined.shape)

    return support_embeddings_combined, support_labels_combined

# ANNOTATIONS DATASET & BATCH SAMPLER

In [8]:
class ProtoDataset(Dataset):
    def __init__(self, embeddings, labels):
        assert embeddings.shape[0] == labels.shape[0]
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]


class PrototypicalBatchSampler(object):
    """
    Yields a batch of indices for episodic training.
    At each iteration, it randomly selects 'classes_per_it' classes and then picks
    'num_samples' samples for each selected class.
    """
    def __init__(self, labels, classes_per_it, num_samples, iterations):
        """
        Args:
            labels (array-like): 1D array or list of labels for the target task.
                                 This should be either the shape labels or the colour labels.
            classes_per_it (int): Number of random classes for each iteration.
            num_samples (int): Number of samples per class (support + query) in each episode.
            iterations (int): Number of iterations (episodes) per epoch.
        """
        self.labels = np.array(labels)
        self.classes_per_it = classes_per_it
        self.sample_per_class = num_samples
        self.iterations = iterations
        
        self.classes, self.counts = np.unique(self.labels, return_counts=True)
        self.classes = torch.LongTensor(self.classes)

        # Create an index matrix of shape (num_classes, max_samples_in_class)
        max_count = max(self.counts)
        self.indexes = np.empty((len(self.classes), max_count), dtype=int)
        self.indexes.fill(-1)
        self.indexes = torch.LongTensor(self.indexes)
        self.numel_per_class = torch.zeros(len(self.classes), dtype=torch.long)

        # Fill in the matrix with indices for each class.
        for idx, label in enumerate(self.labels):
            # Find the row corresponding to this label
            class_idx = (self.classes == label).nonzero(as_tuple=False).item()
            # Find the next available column (where the value is -1)
            pos = (self.indexes[class_idx] == -1).nonzero(as_tuple=False)[0].item()
            self.indexes[class_idx, pos] = idx
            self.numel_per_class[class_idx] += 1

    def __iter__(self):
        """
        Yield a batch of indices for each episode.
        """
        spc = self.sample_per_class
        cpi = self.classes_per_it

        for _ in range(self.iterations):
            batch = torch.LongTensor(cpi * spc)
            # Randomly choose 'classes_per_it' classes
            c_idxs = torch.randperm(len(self.classes))[:cpi]
            for i, class_idx in enumerate(c_idxs):
                s = slice(i * spc, (i + 1) * spc)

                n_avail = self.numel_per_class[class_idx]
                if spc <= n_avail:
                    # enough examples → sample without replacement
                    perm = torch.randperm(n_avail)
                    sample_idxs = perm[:spc]
                else:
                    # too few → sample with replacement
                    sample_idxs = torch.randint(0, n_avail, (spc,), dtype=torch.long)

                batch[s] = self.indexes[class_idx, sample_idxs]

            # Shuffle the batch indices
            batch = batch[torch.randperm(len(batch))]
            yield batch

    def __len__(self):
        return self.iterations

# UNSUPERVISED DATA AND MODEL LOADING

In [9]:
dataset = get_dataset(args)
n_images, c_split = dataset.get_split()

encoder, decoder = dataset.get_backbone()
assert isinstance(encoder, tuple) and len(encoder) == 21, "encoder must be a tuple of 21 elements"

# & Main model
model = get_model(args, encoder, decoder, n_images, c_split)
model.start_optim(args)
check_optimizer_params(model)
loss = model.get_loss(args)

print(dataset)
print("Using Dataset: ", dataset)
print("Using Model: ", model)
print("Using Loss: ", loss)

unsup_train_loader, unsup_val_loader, unsup_test_loader = dataset.get_data_loaders(args=args)

Available datasets: ['mnmath', 'xor', 'clipboia', 'shortmnist', 'restrictedmnist', 'minikandinsky', 'presddoia', 'prekandinsky', 'sddoia', 'clipkandinsky', 'addmnist', 'clipshortmnist', 'boia_original', 'boia_original_embedded', 'clipsddoia', 'boia', 'kandinsky', 'halfmnist']
[PROTO-INFO] Using Prototypical Networks as backbone
Available models: ['promnistltn', 'promnmathcbm', 'sddoiann', 'kandnn', 'sddoiadpl', 'sddoialtn', 'kandslsingledisj', 'presddoiadpl', 'boiann', 'mnistclip', 'prokanddpl', 'promnistdpl', 'kandltnsinglejoint', 'xornn', 'mnistnn', 'mnistslrec', 'kandpreprocess', 'kandsl', 'kandsloneembedding', 'prokandltn', 'kandcbm', 'prokandsl', 'boiacbm', 'kanddpl', 'kandltn', 'xorcbm', 'sddoiaclip', 'kanddplsinglejoint', 'xordpl', 'promnmathdpl', 'bddoiadpldisj', 'sddoiacbm', 'mnistltnrec', 'mnmathcbm', 'mnmathdpl', 'kandclip', 'minikanddpl', 'mnistdpl', 'mnistltn', 'boiadpl', 'boialtn', 'shieldedmnist', 'kandltnsingledisj', 'prokandsloneembedding', 'mnistpcbmdpl', 'mnistcbm', 

## Check Optimizer

In [10]:
all_params = sum(p.numel() for p in model.opt.param_groups[0]['params'])
pnets_params = sum(p.numel() for enc in model.encoder for p in enc.parameters())
mlp_params = sum(p.numel() for p in model.mlp.parameters()) if args.expressive_model else 0

expected_total = pnets_params + mlp_params

assert all_params == expected_total, (
    f"Mismatch in optimizer parameters!\n"
    f"- Backbone params: {pnets_params:,}\n"
    f"- MLP params: {mlp_params:,} (expressive_model={args.expressive_model})\n"
    f"- Total in optimizer: {all_params:,}\n"
    f"→ Expected total: {expected_total:,}"
)

# FETCHING DATA ANNOTATIONS

## Build positive annotation set for each class

In [11]:

# * 1 POSITIVE EXAMPLES COLLECTION =====
pos_examples = {cls_idx: [] for cls_idx in range(21)}
target_per_class = 6    # Desired number of positives per class
debug = True

# Loop over dataset until we collect target_per_class for each class
for batch_idx, batch in enumerate(unsup_train_loader):
    raw_embs = torch.stack(batch['embeddings_raw']).to(model.device)
    attrs = torch.stack(batch['attr_labels']).to(model.device)  # shape [B,21]
    batch_size = attrs.size(0)

    for b in range(batch_size):
        attr_vector = attrs[b].clone().cpu()
        for cls in torch.nonzero(attr_vector).flatten().tolist():
            if len(pos_examples[cls]) >= target_per_class:
                continue
            example = {
                'source_id': (batch_idx, b),
                'images_embeddings_raw': raw_embs[b].detach().cpu().clone(),
                'attr_labels': attr_vector,
                'is_positive': True
            }
            if debug:
                for key, value in example.items():
                    if torch.is_tensor(value):
                        print(f"{key}: {value.shape}")
                    elif isinstance(value, list) and len(value) and torch.is_tensor(value[0]):
                        print(f"{key}: list of {len(value)} tensors, first shape: {value[0].shape}")
                    else:
                        print(f"{key}: {type(value)}")
                debug = False
            pos_examples[cls].append(example)

    if all(len(pos_examples[c]) >= target_per_class for c in range(21)):    break

source_id: <class 'tuple'>
images_embeddings_raw: torch.Size([2048])
attr_labels: torch.Size([21])
is_positive: <class 'bool'>


## Augment positive sets while building negative ones

In [12]:

# * 2: NEGATIVE EXAMPLES & MULTI-LABEL AUGMENTATION =====
neg_examples = {cls_idx: [] for cls_idx in range(21)}

# Allow multi-label augmentation: add any example with attr_labels[i]==1 to pos_examples[i]
for cls in range(21):
    seen_ids = {ex['source_id'] for ex in pos_examples[cls]}
    for other_cls in range(21):
        if other_cls == cls:
            continue
        for ex in pos_examples[other_cls]:
            if ex['attr_labels'][cls] == 1 and ex['source_id'] not in seen_ids:
                new_ex = ex.copy()
                new_ex['is_positive'] = True
                pos_examples[cls].append(new_ex)
                seen_ids.add(ex['source_id'])

# Build negatives: any example that has attr_labels[i]==0 but appears in any pos_examples of other classes
for cls in range(21):
    seen_ids_pos = {ex['source_id'] for ex in pos_examples[cls]}
    for other_cls in range(21):
        if other_cls == cls:
            continue
        for ex in pos_examples[other_cls]:
            if ex['attr_labels'][cls] == 0 and ex['source_id'] not in seen_ids_pos:
                neg_ex = ex.copy()
                neg_ex['is_positive'] = False
                neg_examples[cls].append(neg_ex)

# Ensure no overlap between pos and neg
for cls in range(21):
    assert not set(ex['source_id'] for ex in neg_examples[cls]) & set(ex['source_id'] for ex in pos_examples[cls]), \
        f"Overlap in pos/neg for class {cls}"


## Turn annotations into tensors

In [13]:

# * 3: BUILD EMBEDDING TENSORS AND LABELS =====
dataset_per_class = {}
for cls in range(21):
    examples = pos_examples[cls] + neg_examples[cls]
    emb_list, label_list, extended_label_list = [], [], []
    for ex in examples:
        emb_list.append(ex['images_embeddings_raw'].unsqueeze(0))
        label_list.append(1 if ex['is_positive'] else 0)
        extended_label_list.append(ex['attr_labels'])

    embeddings_tensor = torch.stack(emb_list).to(model.device)                  # [N,1,2048]
    labels_tensor = torch.tensor(label_list, device=model.device)               # [N]
    extended_labels_tensor = torch.stack(extended_label_list).to(model.device)  # [N,21]
    dataset_per_class[cls] = {'embeddings': embeddings_tensor, 'labels': labels_tensor, 'extended_labels': extended_labels_tensor}

        
for cls in range(21):
    print(
        f"Class {cls}: embeddings shape = {dataset_per_class[cls]['embeddings'].shape}, "
        f"labels shape = {dataset_per_class[cls]['labels'].shape}, "
        f"extended labels shape = {dataset_per_class[cls]['extended_labels'].shape}"
    )

Class 0: embeddings shape = torch.Size([184, 1, 2048]), labels shape = torch.Size([184]), extended labels shape = torch.Size([184, 21])
Class 1: embeddings shape = torch.Size([206, 1, 2048]), labels shape = torch.Size([206]), extended labels shape = torch.Size([206, 21])
Class 2: embeddings shape = torch.Size([188, 1, 2048]), labels shape = torch.Size([188]), extended labels shape = torch.Size([188, 21])
Class 3: embeddings shape = torch.Size([204, 1, 2048]), labels shape = torch.Size([204]), extended labels shape = torch.Size([204, 21])
Class 4: embeddings shape = torch.Size([219, 1, 2048]), labels shape = torch.Size([219]), extended labels shape = torch.Size([219, 21])
Class 5: embeddings shape = torch.Size([228, 1, 2048]), labels shape = torch.Size([228]), extended labels shape = torch.Size([228, 21])
Class 6: embeddings shape = torch.Size([230, 1, 2048]), labels shape = torch.Size([230]), extended labels shape = torch.Size([230, 21])
Class 7: embeddings shape = torch.Size([208, 1, 

## *X-Model*: dataloader istantiation

In [14]:

# * 4: BUILD THE DATALOADER FOR THE EXPRESSIVE MODEL =====
if args.expressive_model:
    all_embeddings = torch.cat([per_class['embeddings'] for per_class in dataset_per_class.values()], dim=0)
    all_labels = torch.cat([per_class['extended_labels'] for per_class in dataset_per_class.values()], dim=0)
    all_embeddings = all_embeddings.cpu()
    all_labels     = all_labels.cpu()
    dataset = ProtoDataset(all_embeddings, all_labels)
    x_loader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
else:
    x_loader = None

## *PNets*: Sanity checks

In [15]:

# * 5: CONSISTENCY CHECKS FOR PNETs DATA =====
for cls in range(21):
    emb = dataset_per_class[cls]['embeddings']
    lab = dataset_per_class[cls]['labels']
    # Exact count assertion
    expected_total = len(pos_examples[cls]) + len(neg_examples[cls])
    assert emb.size(0) == expected_total, f"Class {cls} count mismatch: {emb.size(0)} vs {expected_total}"
    # Label values correct
    assert set(lab.tolist()) <= {0,1}, f"Invalid labels for class {cls}"
    # Check that each positive and negative example has correct label
    pos_count = len(pos_examples[cls])
    neg_count = len(neg_examples[cls])
    # positives should be labeled 1 in the first pos_count entries
    for idx in range(pos_count):
        assert lab[idx].item() == 1, f"Positive at wrong pos for class {cls}, idx {idx}"
    # negatives should be labeled 0 in the next neg_count entries
    for idx in range(neg_count):
        assert lab[pos_count + idx].item() == 0, f"Negative at wrong pos for class {cls}, idx {pos_count + idx}"

print("Dataset per class built with explicit flags and no overlaps.")

Dataset per class built with explicit flags and no overlaps.


## *PNets*: Istantiate Batch Samplers for each class

In [16]:

# * 6 : CREATE EPISODIC DATALOADERS FOR PNETs =====
proto_datasets = {}
proto_dataloaders = {}

for cls in range(21):
    proto_data = dataset_per_class[cls]['embeddings']
    proto_labels = dataset_per_class[cls]['labels']
    proto_datasets[cls] = ProtoDataset(proto_data, proto_labels)
    proto_sampler = PrototypicalBatchSampler(
                    labels = proto_labels.cpu().numpy(),
                    classes_per_it = args.classes_per_it,
                    num_samples = args.num_samples,
                    iterations = args.iterations,
                )
    proto_dataloaders[cls] = DataLoader(proto_datasets[cls], batch_sampler=proto_sampler)


# Labels count check for proto_datasets[cls] and proto_dataloaders[cls]
for cls in range(21):
    # Dataset label count
    label_counter_dataset = Counter(proto_datasets[cls].labels.cpu().tolist())
    print(f"Class {cls} - Dataset Label 0 count: {label_counter_dataset[0]}, Label 1 count: {label_counter_dataset[1]}")

    # Dataloader label count
    label_counter_loader = Counter()
    for batch in proto_dataloaders[cls]:
        _, labels = batch
        label_counter_loader.update(labels.tolist())
    print(f"Class {cls} - Dataloader Label 0 count: {label_counter_loader[0]}, Label 1 count: {label_counter_loader[1]}")


# Final Sanity Check
for cls in range(21):
    print(f"Class {cls}: Dataset size = {len(proto_datasets[cls])}, Dataloader batches = {len(proto_dataloaders[cls])}")
    assert len(proto_dataloaders[cls]) == args.iterations, \
        f"Class {cls}: Expected {args.iterations} batches, got {len(proto_dataloaders[cls])}"
    for batch in proto_dataloaders[cls]:
        embeddings, labels = batch
        #print("Batch Embeddings Shape:", embeddings.shape, "Labels Shape:", labes.shape)
        assert embeddings.shape == ((args.num_support + args.num_query) * args.classes_per_it, 1, 2048), \
            f"Embeddings shape mismatch: {embeddings.shape}"
        assert labels.shape == ((args.num_support + args.num_query) * args.classes_per_it,), \
            f"Labels shape mismatch: {labels.shape}"

Class 0 - Dataset Label 0 count: 162, Label 1 count: 22
Class 0 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 1 - Dataset Label 0 count: 193, Label 1 count: 13
Class 1 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 2 - Dataset Label 0 count: 169, Label 1 count: 19
Class 2 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 3 - Dataset Label 0 count: 188, Label 1 count: 16
Class 3 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 4 - Dataset Label 0 count: 209, Label 1 count: 10
Class 4 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 5 - Dataset Label 0 count: 222, Label 1 count: 6
Class 5 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 6 - Dataset Label 0 count: 224, Label 1 count: 6
Class 6 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 7 - Dataset Label 0 count: 192, Label 1 count: 16
Class 7 - Dataloader Label 0 count: 1000, Label 1 count: 1000
Class 8 - Dataset Label 0 count: 229, Label 1 count: 6
Cla

# TRAINING

In [17]:
def train(
        model: MnistDPL, 
        _loss: ADDMNIST_DPL,
        save_path: str, 
        proto_datasets: dict,
        proto_dataloaders: dict,
        train_loader: DataLoader,
        val_loader: DataLoader,
        x_loader: DataLoader,
        args: Namespace,
        seed: int = 0,
        eval_concepts: List[str] = ['green_lights', 'follow_traffic', 'road_clear',
        'traffic_lights', 'traffic_signs', 'cars', 'pedestrians', 'riders', 'others',
        'no_lane_left', 'obstacle_left_lane', 'solid_left_line',
        'on_right_turn_lane', 'traffic_light_right', 'front_car_right', 
        'no_lane_right', 'obstacle_right_lane', 'solid_right_line',
        'on_left_turn_lane', 'traffic_light_left', 'front_car_left']
    ) -> float:

    # for full reproducibility
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.backends.cudnn.enabled = False
    
    # early stopping
    best_f1_cacc = 0.0
    epochs_no_improve = 0

    # scheduler & warmup (if used) for main model
    scheduler = torch.optim.lr_scheduler.ExponentialLR(model.opt, args.exp_decay)
    w_scheduler = None
    if args.warmup_steps > 0:
        w_scheduler = GradualWarmupScheduler(model.opt, 1.0, args.warmup_steps)
    
    # --------------------------------------
    # ^ 1. PROTOTYPICAL NETWORKS SCHEDULERS & OPTIMIZERS
    # --------------------------------------
    proto_opts, proto_schs = {}, {}
    for c, name in enumerate(eval_concepts):
        opt = torch.optim.Adam(model.encoder[c].parameters())
        sch = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.5)
        proto_opts[c], proto_schs[c] = opt, sch

    fprint("\n--- Start of Training ---\n")
    for b in range(len(model.encoder)):
        model.encoder[b].train()
        model.encoder[b].to(model.device)
            
    pNet_loss = PrototypicalLoss(n_support=args.num_support)
    for epoch in range(args.n_epochs):

        for e in range(args.proto_epochs):
        # --------------------------------------
        # ^ 2. PROTOTYPICAL NETWORKS TRAINING
        # --------------------------------------
            print('----------------------------------')
            print('--- Prototypical Networks Training ---')                        
            print(f"Prototypical Networks Training Epoch {e + 1}/{args.proto_epochs}")
            losses, accs = {}, {}
            for k, name in enumerate(eval_concepts):
                dl = proto_dataloaders[k]
                opt, sch = proto_opts[k], proto_schs[k]
                l, a = train_my_prototypical_network(
                    dl, args.iterations,
                    model.encoder[k], opt, pNet_loss
                )
                losses[name], accs[name] = l, a
                sch.step()

            for name in eval_concepts:
                avg_l = sum(losses[name]) / len(losses[name])
                avg_a = sum(accs[name])   / len(accs[name])
                print(f"  {name:>25s} - Loss {avg_l:.4f} | Acc {avg_a:.4f}")

        # --------------------------------------
        # ^ 3. X-MODEL TRAINING
        # --------------------------------------
        if args.expressive_model:
            print('------------------')
            print('--- X-Model Training ---')
            model.to(model.device)
            model.train()
            for i, batch in enumerate(x_loader):
                batch_embeds, batch_labels = batch
                batch_embeds = batch_embeds.to(model.device)
                batch_labels = batch_labels.to(model.device)
                assert batch_labels.shape[1] == model.n_facts, (
                    f"batch_labels shape is {batch_labels.shape}, expected (batch_size, {model.n_facts})"
                )

                support_emb_dict = {}
                for j in range(model.n_facts):
                    emb_s, lab_s = get_per_class_support_set(
                        proto_datasets, pos_examples, class_idx=j, device=model.device
                    )
                    support_emb_dict[j] = (emb_s, lab_s)

                model.opt.zero_grad()
                out_dict = model(batch_embeds, support_emb_dict)
                concept_predictions = out_dict["CS"]
                loss = F.binary_cross_entropy(concept_predictions, batch_labels.float())

                loss.backward()
                model.opt.step()

                progress_bar(i, len(x_loader), epoch, loss.item())

        # --------------------------------------
        # ^ 4. MAIN MODEL TRAINING
        # --------------------------------------
        print('------------------')
        print('--- Main Model Training ---')    
        print(f"Main Model Training Epoch {epoch + 1}/{args.n_epochs}")  
        ys, y_true, cs, cs_true, batch = None, None, None, None, 0
        for i, batch in enumerate(train_loader):
            # ------------------ original embeddings
            images_embeddings = torch.stack(batch['embeddings']).to(model.device)
            attr_labels = torch.stack(batch['attr_labels']).to(model.device)
            class_labels = torch.stack(batch['class_labels'])[:,:-1].to(model.device) # exclude the last column
            # ------------------ my extracted features
            images_embeddings_raw = torch.stack(batch['embeddings_raw']).to(model.device)
            detected_rois = batch['rois']
            detected_rois_feats = batch['roi_feats']
            detection_labels = batch['detection_labels']
            detection_scores = batch['detection_scores']
            assert_inputs(images_embeddings, attr_labels, class_labels,
                    detected_rois_feats, detected_rois, detection_labels,
                    detection_scores, images_embeddings_raw)
            
            support_emb_dict = {}
            for j in range(model.n_facts):
                emb_s, lab_s = get_per_class_support_set(
                    proto_datasets, pos_examples, class_idx=j, device=model.device
                )
                support_emb_dict[j] = (emb_s, lab_s)

            if random.random() > UNS_PERCENTAGE:
                continue  # Skip this batch with probability (1 - percentage)
            
            out_dict = model(images_embeddings_raw, support_emb_dict)
            out_dict.update({"LABELS": class_labels, "CONCEPTS": attr_labels})
                
            model.opt.zero_grad()
            loss, losses = _loss(out_dict, args)
            loss.backward()
            model.opt.step()

            if ys is None:
                    ys = out_dict["YS"]
                    y_true = out_dict["LABELS"]
                    cs = out_dict["pCS"]
                    cs_true = out_dict["CONCEPTS"]
            else:
                ys = torch.concatenate((ys, out_dict["YS"]), dim=0)
                y_true = torch.concatenate((y_true, out_dict["LABELS"]), dim=0)
                cs = torch.concatenate((cs, out_dict["pCS"]), dim=0)
                cs_true = torch.concatenate((cs_true, out_dict["CONCEPTS"]), dim=0)

            progress_bar(i, len(train_loader), epoch, loss.item())
            
        # --------------------------------------
        # ^ 5. Evaluation phase
        # --------------------------------------
        model.eval()
        for b in range(len(model.encoder)): 
            model.encoder[b].eval()

        if debug:
            y_pred = torch.argmax(ys, dim=-1)
            print("Argmax predictions have shape: ", y_pred.shape)

        my_metrics = evaluate_metrics(model, val_loader, args, 
                    support_emb_dict=support_emb_dict,
                    eval_concepts=eval_concepts,)

        loss = my_metrics[0]
        cacc = my_metrics[1]
        yacc = my_metrics[2]
        f1_y = my_metrics[3]
       
        # update at end of the epoch
        if epoch < args.warmup_steps:   w_scheduler.step()
        else:
            scheduler.step()
            if hasattr(_loss, "grade"):
                _loss.update_grade(epoch)

        ### LOGGING ###
        fprint("  ACC C", cacc, "  ACC Y", yacc, "F1 Y", f1_y)
        
        if not args.tuning and cacc > best_f1_cacc:
            print("Saving...")
            # Update best F1 score
            best_f1_cacc = cacc
            epochs_no_improve = 0

            # Save the best model
            torch.save(model.state_dict(), save_path)
            print(f"Saved best model with cacc score: {best_f1_cacc}")
            
        elif cacc <= best_f1_cacc:
            epochs_no_improve += 1

        if epochs_no_improve >= args.patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    fprint("\n--- End of Training ---\n")

    return best_f1_cacc, support_emb_dict
       

## Run Training

In [18]:
print(f"*** Training model with seed {args.seed}")
print("Chosen device:", model.device)
if not os.path.exists(save_path): os.makedirs(save_path, exist_ok=True)
save_folder = os.path.join(save_path, f"{save_model_name}_{args.seed}.pth")
print("Saving model in folder: ", save_folder)

best_f1_c, support_emb_dict = train(
        model=model,
        proto_datasets=proto_datasets,
        proto_dataloaders=proto_dataloaders,
        train_loader=unsup_train_loader,
        val_loader=unsup_val_loader,
        x_loader=x_loader,
        save_path=save_folder,
        _loss=loss,
        args=args,
        seed=SEED,
)
save_model(model, args, args.seed)  # save the model parameters
print(f"*** Finished training model with seed {args.seed} and best CACC score {best_f1_c}")

print("Training finished.")

*** Training model with seed 1
Chosen device: cuda
Saving model in folder:  ../notebook-outputs/bddoia/my_models/dpl/episodic-proto-net-pipeline-1.0-PROVA/dpl_1.pth

--- Start of Training ---

----------------------------------
--- Prototypical Networks Training ---
Prototypical Networks Training Epoch 1/1


100%|██████████| 100/100 [00:00<00:00, 118.63it/s]


               green_lights - Loss 10.6145 | Acc 0.5560
             follow_traffic - Loss 15.5993 | Acc 0.5320
                 road_clear - Loss 13.5651 | Acc 0.5310
             traffic_lights - Loss 11.1642 | Acc 0.5620
              traffic_signs - Loss 5.8869 | Acc 0.8950
                       cars - Loss 12.2961 | Acc 0.6700
                pedestrians - Loss 12.6790 | Acc 0.5660
                     riders - Loss 13.9757 | Acc 0.5560
                     others - Loss 9.6395 | Acc 0.7510
               no_lane_left - Loss 11.6033 | Acc 0.5580
         obstacle_left_lane - Loss 12.4582 | Acc 0.5410
            solid_left_line - Loss 15.5246 | Acc 0.5820
         on_right_turn_lane - Loss 10.4804 | Acc 0.6950
        traffic_light_right - Loss 14.0604 | Acc 0.5470
            front_car_right - Loss 15.2369 | Acc 0.5350
              no_lane_right - Loss 12.9419 | Acc 0.5310
        obstacle_right_lane - Loss 14.3094 | Acc 0.5260
           solid_right_line - Loss 12.6107 | Acc 0

[ 09-04 | 15:37 ] epoch 0: |██████████████████████████████████████████████████| loss: 4.35422659

  ACC C 63.46657458278868   ACC Y 49.18981481481481 F1 Y 46.39600776538742
Saving...
Saved best model with cacc score: 63.46657458278868
----------------------------------
--- Prototypical Networks Training ---
Prototypical Networks Training Epoch 1/1


...

  ACC C 56.007497012615204   ACC Y 59.75477430555556 F1 Y 44.576167305103795
----------------------------------
--- Prototypical Networks Training ---
Prototypical Networks Training Epoch 1/1


100%|██████████| 100/100 [00:00<00:00, 105.73it/s]


               green_lights - Loss 0.6838 | Acc 0.6400
             follow_traffic - Loss 0.6927 | Acc 0.6150
                 road_clear - Loss 0.6931 | Acc 0.4910
             traffic_lights - Loss 0.0462 | Acc 0.9890
              traffic_signs - Loss 0.0151 | Acc 0.9960
                       cars - Loss 0.0005 | Acc 1.0000
                pedestrians - Loss 0.6931 | Acc 0.5000
                     riders - Loss 0.0031 | Acc 1.0000
                     others - Loss 0.0005 | Acc 1.0000
               no_lane_left - Loss 0.6931 | Acc 0.5610
         obstacle_left_lane - Loss 0.0006 | Acc 1.0000
            solid_left_line - Loss 0.0010 | Acc 1.0000
         on_right_turn_lane - Loss 0.8034 | Acc 0.6480
        traffic_light_right - Loss 0.6931 | Acc 0.5530
            front_car_right - Loss 0.7163 | Acc 0.4850
              no_lane_right - Loss 0.6931 | Acc 0.5000
        obstacle_right_lane - Loss 0.6931 | Acc 0.5000
           solid_right_line - Loss 0.6931 | Acc 0.5000
          

[ 09-04 | 16:21 ] epoch 15: |██████████████████████████████████████████████████| loss: 2.75822139

  ACC C 62.414573629697166   ACC Y 59.450954861111114 F1 Y 44.30103823512157
Early stopping triggered after 16 epochs.

--- End of Training ---

*** Finished training model with seed 1 and best CACC score 63.46657458278868
Training finished.


# EVALUATION

In [19]:
# * Evaluate the model and save metrics
def evaluate_my_model(model: MnistDPL, 
        save_path: str, 
        test_loader: DataLoader,
        eval_concepts: List[str],
        args: Namespace,
        support_embeddings=None,
    ):
    
    if args.model == 'probddoiadpl':
        assert support_embeddings is not None, "Support embeddings must be provided for probddoiadpl model evaluation."
        my_metrics = evaluate_metrics(model, test_loader, args,
                        support_emb_dict=support_embeddings, 
                        eval_concepts=eval_concepts
        )
    else:
        my_metrics = evaluate_metrics(model, test_loader, args, 
                        eval_concepts=eval_concepts
        )

    loss = my_metrics[0]
    cacc = my_metrics[1]
    yacc = my_metrics[2]
    f1_y = my_metrics[3]
    f1_micro = my_metrics[4]
    f1_weight = my_metrics[5]
    f1_bin = my_metrics[6]

    metrics_log_path = save_path.replace(".pth", "_metrics.log")
    
    all_concepts = [ 'Green Traffic Light', 'Follow Traffic', 'Road Is Clear',
        'Red Traffic Light', 'Traffic Sign', 'Obstacle Car', 'Obstacle Pedestrian', 'Obstacle Rider', 'Obstacle Others',
        'No Lane On The Left',  'Obstacle On The Left Lane',  'Solid Left Line',
                'On The Right Turn Lane', 'Traffic Light Allows Right', 'Front Car Turning Right', 
        'No Lane On The Right', 'Obstacle On The Right Lane', 'Solid Right Line',
                'On The Left Turn Lane',  'Traffic Light Allows Left',  'Front Car Turning Left' 
    ]
    aggregated_metrics = [
            'F1 - Binary', 'F1 - Macro', 'F1 - Micro', 'F1 - Weighted',
            'Precision - Binary', 'Precision - Macro', 'Precision - Micro', 'Precision - Weighted',
            'Recall - Binary', 'Recall - Macro', 'Recall - Micro', 'Recall - Weighted',
            'Balanced Accuracy'
    ]

    sums = [0.0] * len(aggregated_metrics)
    num_concepts = len(all_concepts)
    with open(metrics_log_path, "a") as log_file:
        log_file.write(f"ACC C: {cacc}, ACC Y: {yacc}\n\n")
        log_file.write(f"F1 Y - Macro: {f1_y}, F1 Y - Micro: {f1_micro}, F1 Y - Weighted: {f1_weight}, F1 Y - Binary: {f1_bin} \n\n")

        def write_metrics(class_name, offset):
            print(f"Reporting Metrics for {class_name} in {metrics_log_path}")
            log_file.write(f"{class_name.upper()}\n")
            for idx, metric_name in enumerate(aggregated_metrics):
                value = my_metrics[offset + idx]
                sums[idx] += value
                log_file.write(f"  {metric_name:<18} {value:.4f}\n")
            log_file.write("\n")

        i = 7
        for concept in all_concepts:
            write_metrics(concept, i)
            i += len(aggregated_metrics)

        log_file.write("**MEAN ACROSS ALL CONCEPTS**\n")
        for idx, metric_name in enumerate(aggregated_metrics):
            mean_value = sums[idx] / num_concepts
            log_file.write(f"  {metric_name:<18} {mean_value:.4f}\n")
        log_file.write("\n")


    assert len(my_metrics) == 7 + len(all_concepts) * len(aggregated_metrics), \
        f"Expected {7 + len(all_concepts) * len(aggregated_metrics)} metrics, but got {len(my_metrics)}"
    
    if args.model == 'probddoiadpl':
        y_true, c_true, y_pred, c_pred, p_cs, p_ys, p_cs_all, p_ys_all = (
            evaluate_metrics(model, test_loader, args,
                        support_emb_dict=support_embeddings, 
                        eval_concepts=eval_concepts,
                        last=True
                )
        )
    else:
        y_true, c_true, y_pred, c_pred, p_cs, p_ys, p_cs_all, p_ys_all = (
            evaluate_metrics(model, test_loader, args,
                        eval_concepts=eval_concepts,
                        last=True
                )
        )
    
    y_labels = ["stop", "forward", "left", "right"]
    concept_labels = [
        "green_light",      
        "follow",           
        "road_clear",       
        "red_light",        
        "traffic_sign",     
        "car",              
        "person",           
        "rider",            
        "other_obstacle",   
        "left_lane",
        "left_green_light",
        "left_follow",
        "no_left_lane",
        "left_obstacle",
        "letf_solid_line",
        "right_lane",
        "right_green_light",
        "right_follow",
        "no_right_lane",
        "right_obstacle",
        "right_solid_line",
    ]

    plot_multilabel_confusion_matrix(y_true, y_pred, y_labels, "Labels", save_path=save_path)
    cfs = plot_actions_confusion_matrix(c_true, c_pred, "Concepts", save_path=save_path)
    cf = plot_multilabel_confusion_matrix(c_true, c_pred, concept_labels, "Concepts", save_path=save_path)
    print("Concept collapse", 1 - compute_coverage(cf))

    with open(metrics_log_path, "a") as log_file:
        for key, value in cfs.items():
            log_file.write(f"Concept collapse: {key}, {1 - compute_coverage(value):.4f}\n")
            log_file.write("\n")

    fprint("\n--- End of Evaluation ---\n")

## Run Evaluation

In [20]:
# Initialize the model object
model = get_model(args, encoder, decoder, n_images, c_split)

# Load the model state dictionary into the model object
model_state_dict = torch.load(save_folder)
model.load_state_dict(model_state_dict)

# Evaluate the model
evaluate_my_model(
    model=model, 
    save_path=save_folder, 
    test_loader=unsup_test_loader,
    eval_concepts=['green_lights', 'follow_traffic', 'road_clear',
        'traffic_lights', 'traffic_signs', 'cars', 'pedestrians', 'riders', 'others',
        'no_lane_left', 'obstacle_left_lane', 'solid_left_line',
        'on_right_turn_lane', 'traffic_light_right', 'front_car_right',
        'no_lane_right', 'obstacle_right_lane', 'solid_right_line',
        'on_left_turn_lane', 'traffic_light_left', 'front_car_left'],
    args=args,
    support_embeddings=support_emb_dict
)

Available models: ['promnistltn', 'promnmathcbm', 'sddoiann', 'kandnn', 'sddoiadpl', 'sddoialtn', 'kandslsingledisj', 'presddoiadpl', 'boiann', 'mnistclip', 'prokanddpl', 'promnistdpl', 'kandltnsinglejoint', 'xornn', 'mnistnn', 'mnistslrec', 'kandpreprocess', 'kandsl', 'kandsloneembedding', 'prokandltn', 'kandcbm', 'prokandsl', 'boiacbm', 'kanddpl', 'kandltn', 'xorcbm', 'sddoiaclip', 'kanddplsinglejoint', 'xordpl', 'promnmathdpl', 'bddoiadpldisj', 'sddoiacbm', 'mnistltnrec', 'mnmathcbm', 'mnmathdpl', 'kandclip', 'minikanddpl', 'mnistdpl', 'mnistltn', 'boiadpl', 'boialtn', 'kandltnsingledisj', 'prokandsloneembedding', 'mnistpcbmdpl', 'mnistcbm', 'probddoiadpl', 'mnistpcbmsl', 'mnistpcbmltn', 'kanddplsingledisj', 'mnistsl', 'kandslsinglejoint', 'mnistdplrec', 'cvae', 'cext', 'mnmathnn', 'promnistsl']
Reporting Metrics for Green Traffic Light in ../notebook-outputs/bddoia/my_models/dpl/episodic-proto-net-pipeline-1.0-PROVA/dpl_1_metrics.log
Reporting Metrics for Follow Traffic in ../noteb